## 1. Data analysis

In [1]:
import pandas as pd

In [3]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [6]:
print("Train Data Overview:")
print(train_data.head())

print("Test Data Overview:")
print(test_data.head())

Train Data Overview:
   Id  Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  \
0   1       2596      51      3                               258   
1   2       2590      56      2                               212   
2   3       2804     139      9                               268   
3   4       2785     155     18                               242   
4   5       2595      45      2                               153   

   Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                               0                              510   
1                              -6                              390   
2                              65                             3180   
3                             118                             3090   
4                              -1                              391   

   Hillshade_9am  Hillshade_Noon  Hillshade_3pm  ...  Soil_Type32  \
0            221             232            148  ...            0   
1    

In [7]:
print("Missing values in train data:")
print(train_data.isnull().sum())

Missing values in train data:
Id                                    0
Elevation                             0
Aspect                                0
Slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Wilderness_Area1                      0
Wilderness_Area2                      0
Wilderness_Area3                      0
Wilderness_Area4                      0
Soil_Type1                            0
Soil_Type2                            0
Soil_Type3                            0
Soil_Type4                            0
Soil_Type5                            0
Soil_Type6                            0
Soil_Type7                            0
Soil_Type8                            0
Soil_Type9                            0
Soil_Type1

In [8]:
print("Missing values in test data:")
print(test_data.isnull().sum())

Missing values in test data:
Id                                    0
Elevation                             0
Aspect                                0
Slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Wilderness_Area1                      0
Wilderness_Area2                      0
Wilderness_Area3                      0
Wilderness_Area4                      0
Soil_Type1                            0
Soil_Type2                            0
Soil_Type3                            0
Soil_Type4                            0
Soil_Type5                            0
Soil_Type6                            0
Soil_Type7                            0
Soil_Type8                            0
Soil_Type9                            0
Soil_Type10

In [9]:
print("Train data description:")
print(train_data.describe())

Train data description:
                Id     Elevation        Aspect         Slope  \
count  15120.00000  15120.000000  15120.000000  15120.000000   
mean    7560.50000   2749.322553    156.676653     16.501587   
std     4364.91237    417.678187    110.085801      8.453927   
min        1.00000   1863.000000      0.000000      0.000000   
25%     3780.75000   2376.000000     65.000000     10.000000   
50%     7560.50000   2752.000000    126.000000     15.000000   
75%    11340.25000   3104.000000    261.000000     22.000000   
max    15120.00000   3849.000000    360.000000     52.000000   

       Horizontal_Distance_To_Hydrology  Vertical_Distance_To_Hydrology  \
count                      15120.000000                    15120.000000   
mean                         227.195701                       51.076521   
std                          210.075296                       61.239406   
min                            0.000000                     -146.000000   
25%                     

In [10]:
print(train_data['Cover_Type'].value_counts())

Cover_Type
5    2160
2    2160
1    2160
7    2160
3    2160
6    2160
4    2160
Name: count, dtype: int64


## 2. Model training

### 2.1. Random Forest

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [13]:
X = train_data.drop(columns=['Cover_Type'])
y = train_data['Cover_Type']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [15]:
# Data scaling if needed
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
# Model build
model = RandomForestClassifier(random_state=1)
model.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=1)

In [26]:
# Predicting and accuracy
y_pred_rf = model.predict(X_test_scaled)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf:.4f}")

Random Forest Accuracy: 0.8737


### 2.2. Gradient Boosting

In [27]:
from sklearn.ensemble import GradientBoostingClassifier

In [28]:
# Model build
gb_model = GradientBoostingClassifier(random_state=1)
gb_model.fit(X_train_scaled, y_train)

GradientBoostingClassifier(random_state=1)

In [29]:
# Predicting and accuracy
y_pred_gb = gb_model.predict(X_test_scaled)
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print(f"Gradient Boosting Accuracy: {accuracy_gb:.4f}")

Gradient Boosting Accuracy: 0.8128


In [30]:
# The result is kind of bad, so let's improve it
from sklearn.model_selection import GridSearchCV

In [32]:
# Defining the grid of hyperparameters for the research
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

In [33]:
# Initialzing the GB model
gb_model = GradientBoostingClassifier(random_state=1)

# GridSearch optimizing
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid,
                          cv=3, n_jobs=-1, verbose=2, scoring='accuracy')

# Training with hyperparameters fit
grid_search.fit(X_train_scaled, y_train)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation Parameters: {grid_search.best_score_}")

Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best Parameters: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 200, 'subsample': 1.0}
Best Cross-Validation Parameters: 0.8576388888888888


In [34]:
# Model score with best parameters
best_gb_model = grid_search.best_estimator_
y_pred_best_gb = best_gb_model.predict(X_test_scaled)
accuracy_best_gb = accuracy_score(y_test, y_pred_best_gb)
print(f"Gradient Boosting Accuracy with best parameters: {accuracy_best_gb:.4f}")

Gradient Boosting Accuracy with best parameters: 0.8740


### 2.3. Voting Classifier

In [36]:
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier

In [37]:
# Model initializing
rf_model = RandomForestClassifier(random_state=1)
gb_model = GradientBoostingClassifier(random_state=1)
xgb_model = XGBClassifier(random_state=1)

In [39]:
# Creating an ensemble Voting Classifier
voting_clf = VotingClassifier(estimators=[
    ('rf', rf_model),
    ('gb', gb_model),
    ('xgb', xgb_model)],
    voting='hard')

In [40]:
# Training the ensemble
voting_clf.fit(X_train_scaled, y_train)

# Accuracy score
y_pred_voting = voting_clf.predict(X_test_scaled)
accuracy_voting = accuracy_score(y_test, y_pred_voting)
print(f"Voting Classifier Accuracy: {accuracy_voting:.4f}")

Voting Classifier Accuracy: 0.8697


## 3. Predicting on the test data

In [57]:
# Scaling the test data
test_data_scaled = scaler.transform(test_data)

# Predicting on the test dataset
test_predictions = best_gb_model.predict(test_data_scaled)

# Creating a submission file for Kaggle
submission = pd.DataFrame({
    'Id': test_data['Id'],
    'Cover_Type': test_predictions
})

# Saving the file
submission.to_csv('submission.csv', index=False)